#### Ideal Quantum SVM - Spambase - Small Subset Dataset

In [ ]:
# Check Qiskit, Qiskit Aer versions, Qiskit Machine Learning - Versions
import qiskit
import qiskit_aer
print(qiskit.__version__)
print("Aer:", qiskit_aer.__version__)

1.4.4
Aer: 0.17.2


In [ ]:
# To ensure reproducibility of results
from qiskit_machine_learning.utils import algorithm_globals
algorithm_globals.random_seed = 12345

In [ ]:
# Imports
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

# Qiskit Imports
# Definine quantum kernel
# Use the FidelityQuantumKernel class 

from qiskit.circuit.library import ZZFeatureMap
from qiskit.primitives import StatevectorSampler as Sampler
from qiskit_machine_learning.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel

In [ ]:
# --- Import Spambase Column Names ---
spambase_columns = [
    "word_freq_make",
    "word_freq_address",
    "word_freq_all",
    "word_freq_3d",
    "word_freq_our",
    "word_freq_over",
    "word_freq_remove",
    "word_freq_internet",
    "word_freq_order",
    "word_freq_mail",
    "word_freq_receive",
    "word_freq_will",
    "word_freq_people",
    "word_freq_report",
    "word_freq_addresses",
    "word_freq_free",
    "word_freq_business",
    "word_freq_email",
    "word_freq_you",
    "word_freq_credit",
    "word_freq_your",
    "word_freq_font",
    "word_freq_000",
    "word_freq_money",
    "word_freq_hp",
    "word_freq_hpl",
    "word_freq_george",
    "word_freq_650",
    "word_freq_lab",
    "word_freq_labs",
    "word_freq_telnet",
    "word_freq_857",
    "word_freq_data",
    "word_freq_415",
    "word_freq_85",
    "word_freq_technology",
    "word_freq_1999",
    "word_freq_parts",
    "word_freq_pm",
    "word_freq_direct",
    "word_freq_cs",
    "word_freq_meeting",
    "word_freq_original",
    "word_freq_project",
    "word_freq_re",
    "word_freq_edu",
    "word_freq_table",
    "word_freq_conference",
    "char_freq_;",
    "char_freq_(",
    "char_freq_[",
    "char_freq_!",
    "char_freq_$",
    "char_freq_#",
    "capital_run_length_average",
    "capital_run_length_longest",
    "capital_run_length_total",
    # finally the target label column:
    "label"
]

# --- 1. Load the Spambase Dataset ---
file_path = r'C:\Users\User\Documents\MyProjects\FYP_ResearchProject\data\spambase\spambase.data'
df = pd.read_csv(file_path, header=None, names=spambase_columns)
df.drop_duplicates(inplace=True)


In [ ]:
# 3. Separate features and target
X = df.drop('label', axis=1) # Columns axis 1, Rows axis 2 - just additional info
y = df['label']

# Now got : 
# Features - X
# Target - y

In [ ]:
# Data splitting
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# test_size - 0.3 means 30% as test set
# random_state - ensures the random shuffling is the same every time the code runs
# if its random, the result will be different and other people might ended up getting different results as well
# stratify=y - nsures fairness when comparing classical SVM vs QSVM, especially if dataset is imbalanced (like more spam than non-spam emails).
# Look at the labels in y, calculate the percentage of each class (like 80% Class A and 20% Class B), and make sure the new training set and 
# testing set both keep that exact same 80/20 ratio.

In [ ]:
# Scaling and PCA

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

n_components = 4
pca = PCA(n_components=n_components)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

'''
StandardScaler - as mentioned before, it normalizes the feature to mean 0, std 1
PCA - reduces dimensionality to 4 principal components, 4 also because to match with the 4 qubit feature map
PCA - also why choose change into 4 components is to match 4 qubits of ZZFeatureMap
Also (Binary Classification) is just the classification between two classes. It does nothing to amount of qubits

Data leakage - information from test sets sneaks into the training process, makes model look better because it looks like it seen some information

Additional Info:
Why only scale and PCA the features x and not labels y
- X is because they are numerical
- So need better format of features (same range so they dont dominate), and reduce number of features to match the number of qubits

- Y are class identifiers
- Not features 
- If scaled they it destroys their meaning

Summary :
Scale + PCA → features (X)
Do not touch → labels (y)

'''

'\nStandardScaler - as mentioned before, it normalizes the feature to mean 0, std 1\nPCA - reduces dimensionality to 4 principal components, 4 also because to match with the 4 qubit feature map\nPCA - also why choose change into 4 components is to match 4 qubits of ZZFeatureMap\nAlso (Binary Classification) is just the classification between two classes. It does nothing to amount of qubits\n\nData leakage - information from test sets sneaks into the training process, makes model look better because it looks like it seen some information\n\nAdditional Info:\nWhy only scale and PCA the features x and not labels y\n- X is because they are numerical\n- So need better format of features (same range so they dont dominate), and reduce number of features to match the number of qubits\n\n- Y are class identifiers\n- Not features \n- If scaled they it destroys their meaning\n\nSummary :\nScale + PCA → features (X)\nDo not touch → labels (y)\n\n'

In [ ]:
# Creating a Medium-Sized Subset of Spambase Dataset for Faster Execution
# Because took too long to run on full dataset
subset_size = 600 
X_train_subset = X_train_pca[:subset_size]
y_train_subset = y_train[:subset_size]

# Same goes for test set
X_test_subset = X_test_pca[:subset_size]
y_test_subset = y_test[:subset_size]   

# Check first
print(f"Using a subset of {subset_size} samples for training and testing.")
print(f"Training subset shape: {X_train_subset.shape}\n")
print(f"Testing subset shape: {X_test_subset.shape}\n")

Using a subset of 600 samples for training and testing.
Training subset shape: (600, 4)

Testing subset shape: (600, 4)



##### Quantum Kernel Implementation

In [ ]:
# Quantum Kernel Setup
fm = ZZFeatureMap(feature_dimension=n_components, reps=2, entanglement='linear')
sampler = Sampler()
fidelity = ComputeUncompute(sampler=sampler)
qkernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=fm)

In [ ]:
# Compute those Kernel Matrices
print("Calculating training kernel matrix for the training subset...")
start_time = time.time()
matrix_train = qkernel.evaluate(x_vec = X_train_subset)
end_time = time.time()
print(f"Training kernel matrix calculated in {end_time - start_time:.2f} seconds.")

# For test set
print("Calculating testing kernel matrix for the testing subset...")
start_time = time.time()
matrix_test = qkernel.evaluate(x_vec = X_test_subset, y_vec = X_train_subset)
end_time = time.time()
print(f"Test kernel matrix calculated in {end_time - start_time:.2f} seconds.\n")


Calculating training kernel matrix for the training subset...
Training kernel matrix calculated in 2171.46 seconds.
Calculating testing kernel matrix for the testing subset...
Test kernel matrix calculated in 1743.46 seconds.



In [ ]:
# Classification with SVC
print("--- Training QSVM with Precomputed Kernel (Spambase Subset) ---")
qsvm = SVC(kernel='precomputed')

# Grid Search for Hyperparameter C
param_grid = {'C': [0.1, 1, 10, 100, 1000]}
grid_search = GridSearchCV(qsvm, param_grid, cv=5, verbose=1)
grid_search.fit(matrix_train, y_train_subset)

print(f"\nBest parameters found: {grid_search.best_params_}")
best_qsvm = grid_search.best_estimator_

--- Training QSVM with Precomputed Kernel (Spambase Subset) ---
Fitting 5 folds for each of 5 candidates, totalling 25 fits

Best parameters found: {'C': 0.1}


##### Model Evaluation

In [ ]:
# Evaluation
# For train data
y_train_pred = best_qsvm.predict(matrix_test)
train_accuracy = accuracy_score(y_train_subset, y_train_pred)

# For test data
y_test_pred = best_qsvm.predict(matrix_test)
test_accuracy = accuracy_score(y_test_subset, y_test_pred)

# Calculate the generalization gap
generalization_gap = abs(train_accuracy - test_accuracy)


# Print all results 
print(f"\n--- Ideal QSVM Evaluation (Spambase Subset of {subset_size}) ---")
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy:     {test_accuracy:.4f}")
print(f"Generalization Gap: {generalization_gap:.4f}")
print("\nClassification Report (Test Set):")
print(classification_report(y_test_subset, y_test_pred, zero_division=0))



--- Ideal QSVM Evaluation (Spambase Subset of 600) ---
Training Accuracy: 0.6100
Test Accuracy:     0.5850
Generalization Gap: 0.0250

Classification Report (Test Set):
              precision    recall  f1-score   support

           0       0.58      1.00      0.74       351
           1       0.00      0.00      0.00       249

    accuracy                           0.58       600
   macro avg       0.29      0.50      0.37       600
weighted avg       0.34      0.58      0.43       600

